# MS&E 234 Project

### Data cleaning

In [ ]:
import pandas as pd
import re
from random import sample
import itertools

### Load data

In [ ]:
df_raw = pd.DataFrame()
for i in range(1, 4+1):
    df_raw = pd.concat([df_raw, pd.read_csv(f'../netflix-prize-kaggle-data/combined_data_{i}.txt',
        header=None,
        names=['CustomerID', 'Rating', 'Date'])])

df_raw

### Subset data as in paper (Calandrino 2011)

In [ ]:
df = df_raw
print(f'Entire dataset: {df.shape[0]} ratings; {df.CustomerID.nunique()} users')

# Sample 10000 users
df = df[df['CustomerID'].isin(sample(df['CustomerID'].unique().tolist(), 10000))]
print(f"Subset of 10000 users: {df.shape[0]} ratings; {df['CustomerID'].nunique()} users")

# Assume each user makes a random 50% of transactions public
idxs = df[['CustomerID']].reset_index().groupby('CustomerID').agg({'index':lambda x: list(x)}).to_numpy().tolist()
drop_idxs = []
for i in range(len(idxs)):
    drop_idxs.append(sample(idxs[i][0], len(idxs[i][0]) // 2))
drop_idxs = list(itertools.chain(*drop_idxs))
df = df.drop(drop_idxs)
print(f'After making 50% private: {df.shape[0]} ratings; {df.CustomerID.nunique()} users')

# Only consider users with at least 100 public transactions
df = df[df['CustomerID'].isin(df['CustomerID'].value_counts()[df['CustomerID'].value_counts() > 100].index)]
print(f'Filter for users with >= 100 public transactions: {df.shape[0]} ratings; {df.CustomerID.nunique()} users')

# Subset the data to analyze only ratings from July 2005
df = df[df['Date'].apply(lambda x: type(x) == str and bool(re.match(r'2005-07.*', x)))]
print(f'Only July 2005: {df.shape[0]} ratings; {df["CustomerID"].nunique()} users')

### TODO

In [ ]:
# Sanity check: there are ~1570 such users
# Sanity check: there are around 1510 transactions during the period in question: TODO there are way more transactions - how do we get only ~1 transaction per user per month, when we are filtering for users that recommend many movies? Seems plausible that these users are making ~20 transactions/month...
# Restrict the attack to (customer, date) pairs in which the customer made 5 or fewer transactions: TODO I think this means only compute prediction accuracy on these pairs, rather than removing them before training